In [2]:
import strym
from strym import strymread
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymysql
import re

In [3]:
m_host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com'
m_port =29563
m_user='capstone'
m_password='capstone123!!!'
m_db='circle_database_version3'

In [4]:
#get basic info for filename
def get_vin(file):
    vin_pattern = r"\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}_([A-Z0-9]*)_CAN_Messages.csv"
    vin = re.findall(vin_pattern,file)[0]
    return vin
def get_file_prefix(file):
    data_pattern = r"(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2})_[A-Z0-9]*_CAN_Messages.csv"
    date = re.findall(data_pattern,file)[0]
    return date
#prepare data for insert
def remove_speed_bus0(r):
    slice_lable = (
        r.speed()[['Time']].apply(tuple, axis=1)
        .isin(r.speed_raw(0)[['Time']].apply(tuple, axis=1)
              .to_list()
             )
    )
    return r.speed()[~slice_lable]
def get_fact_speed(drive_id,r):
    speed_cleaned=remove_speed_bus0(r)
    df_speed = pd.DataFrame()
    df_speed['elapsed_time']=speed_cleaned['Time']
    df_speed['speed']=speed_cleaned['Message']
    df_speed['drive_id'] =drive_id
    return df_speed
def get_fact_acceleration(drive_id,r):
    accelx = r.accelx()
    df_accelx = pd.DataFrame()
    df_accelx['elapsed_time'] = accelx['Time']
    df_accelx['acceleration'] = accelx['Message']
    df_accelx['drive_id'] = drive_id
    return df_accelx
def get_fact_lead_distance(drive_id,r):
    lead_distance = r.lead_distance()
    df_lead_distance = pd.DataFrame()
    df_lead_distance['elapsed_time'] = lead_distance['Time']
    df_lead_distance['lead_distance'] = lead_distance['Message']
    df_lead_distance['drive_id'] = drive_id
    return df_lead_distance
def get_fact_relative_vel(drive_id,r):
    relative_vel = r.relative_leadervel()
    df_relative_vel=pd.DataFrame()
    df_relative_vel['elapsed_time'] = relative_vel['Time']
    df_relative_vel['relative_vel'] = relative_vel['Message']
    df_relative_vel = r.remove_duplicates(df_relative_vel)
    df_relative_vel['drive_id'] = drive_id
    return df_relative_vel
def get_fact_acc_status(drive_id,r):
    acc_status = r.acc_state()
    acc_status=acc_status[acc_status['Bus']!=0]
    df_acc_status = pd.DataFrame()
    df_acc_status['elapsed_time'] = acc_status['Time']
    df_acc_status['acc_status'] = acc_status['Message']
    df_acc_status['drive_id'] = drive_id
    return df_acc_status
#use VIN to get vehicle_id
def get_vehicle_id(vin):
    db = pymysql.connect(host=m_host,port =m_port,user=m_user,password=m_password,db=m_db)
    cursor = db.cursor()
    sql ="""SELECT id FROM dim_vehicle where vin = %s;"""
    cursor.execute(sql,vin)
    results = cursor.fetchall()
    if len(results)==0:
        #insert new colunms
        sql ="""INSERT INTO dim_vehicle VALUES (null,%s,null,null,null,null)"""
        cursor.execute(sql,vin)
        db.commit()
        vehicle_id = cursor.lastrowid
    else:
        vehicle_id = results[0][0]
    db.close()
    return vehicle_id
def get_dim_drive_record_with_filename(filename):
    db = pymysql.connect(host=m_host,port =m_port,user=m_user,password=m_password,db=m_db)
    cursor = db.cursor()
    file_prefix = get_file_prefix(filename)
    vehicle_id = get_vehicle_id(get_vin(filename))
    sql = """SELECT id FROM dim_drive where file_prefix REGEXP %s and vehicle_id = %s """
    data=(file_prefix,vehicle_id)
    cursor.execute(sql,data)
    results = cursor.fetchall()      
    if len(results)==0:
        sql = """INSERT INTO dim_drive VALUES (null,%s,%s,null,null,null,null)""" 
        data=(vehicle_id,file_prefix)
        try:  
            cursor.execute(sql,data)
            db.commit()
            return cursor.lastrowid
        except pymysql.Error as e:
            print(e.args[0], e.args[1])
            db.rollback()     
    
    else:
        return results[0][0]
#batch_update
#need transaction
def batch_update(df,table,username,password,server,database):
    from sqlalchemy import create_engine
    import sqlalchemy
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(username,password,server,database)) 
    con = engine.connect()
    try:
        res = df.to_sql(name=table, con=con, if_exists='append', index=False)
        print(res)
    except pymysql.err.IntegrityError as e:
        print(e)
        pass
    except sqlalchemy.exc.IntegrityError as e:
        print(e)
        pass
#we want to insert records into fact_speed
def insert_record_to_fact_speed(file):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    df_speed = get_fact_speed(drive_id,r)
    print(df_speed.head(3))
    batch_update(df_speed,'fact_speed',m_user,m_password,m_host+':'+str(m_port),m_db)
#we want to insert records into fact_acceleration
def insert_record_to_fact_acceleration(file):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    df_acceleration = get_fact_acceleration(drive_id,r)
    print(df_acceleration.head(3))
    batch_update(df_acceleration,'fact_acceleration',m_user,m_password,m_host+':'+str(m_port),m_db)
#we want to insert records into fact_lead_distance
def insert_record_to_fact_lead_distance(file):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    df_lead_distance = get_fact_lead_distance(drive_id,r)
    print(df_lead_distance.head(3))
    batch_update(df_lead_distance,'fact_lead_distance',m_user,m_password,m_host+':'+str(m_port),m_db)
def insert_record_to_fact_relative_vel(file):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    df_relative_vel= get_fact_relative_vel(drive_id,r)
    print(df_relative_vel.head(3))
    batch_update(df_relative_vel,'fact_relative_vel',m_user,m_password,m_host+':'+str(m_port),m_db)
def insert_record_to_fact_acc_status(file):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    df_acc_status= get_fact_acc_status(drive_id,r)
    print(df_acc_status.shape[0])
    print(df_acc_status.head(3))
    batch_update(df_acc_status,'fact_acc_status',m_user,m_password,m_host+':'+str(m_port),m_db)
def insert_record_to_fact_feature(file,table_name):
    r =strymread(csvfile=file)
    drive_id = get_dim_drive_record_with_filename(file)
    f = dict_tablenames[table_name]
    df = f(drive_id,r)
    print(df.head(3))
    batch_update(df,table_name,m_user,m_password,m_host+':'+str(m_port),m_db)

In [5]:
dict_tablenames={
    'fact_speed':get_fact_speed,
    'fact_acceleration':get_fact_acceleration,
    'fact_lead_distance':get_fact_lead_distance,
    'fact_relative_vel':get_fact_relative_vel,
    'fact_acc_status':get_fact_acc_status
}

In [8]:
file = '2021-04-06-15-13-00_2T3Y1RFV8KC014025_CAN_Messages.csv'

In [9]:
insert_record_to_fact_feature(file,'fact_speed')
insert_record_to_fact_feature(file,'fact_acceleration')
insert_record_to_fact_feature(file,'fact_lead_distance')
insert_record_to_fact_feature(file,'fact_relative_vel')
insert_record_to_fact_feature(file,'fact_acc_status')

[2022_03_05_20_58_46] (root) INFO: Vehicle model infered is toyota-rav4-2019
                               elapsed_time  speed  drive_id
Clock                                                       
2021-04-06 15:13:01.248377088  1.617722e+09    0.0         2
2021-04-06 15:13:01.249704960  1.617722e+09    0.0         2
2021-04-06 15:13:01.250845952  1.617722e+09    0.0         2
None
[2022_03_05_20_59_23] (root) INFO: Vehicle model infered is toyota-rav4-2019
                               elapsed_time  acceleration  drive_id
Clock                                                              
2021-04-06 15:13:01.167280896  1.617722e+09           0.0         2
2021-04-06 15:13:01.169508096  1.617722e+09           0.0         2
2021-04-06 15:13:01.173309184  1.617722e+09           0.0         2
None
[2022_03_05_21_00_01] (root) INFO: Vehicle model infered is toyota-rav4-2019
                               elapsed_time  lead_distance  drive_id
Clock                                        

In [46]:
insert_record_to_fact_relative_vel(file)

[2022_02_28_11_20_07] (root) INFO: Vehicle model infered is toyota-rav4-2019
                               elapsed_time  relative_vel  drive_id
Clock                                                              
2021-07-28 17:10:54.812594944  1.627492e+09         0.000         1
2021-07-28 17:10:54.813356032  1.627492e+09         0.000         1
2021-07-28 17:10:54.815834112  1.627492e+09         0.075         1
None


In [20]:
#serach
db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',port =29563,user='capstone',password='capstone123!!!',db='circle_database_version3')
cursor = db.cursor()
sql = "SELECT * FROM fact_relative_vel" 
cursor.execute(sql)
# Fetch all the rows in a list of lists.
results = cursor.fetchall()
db.close()

In [67]:
file='2021_07_26_22_45_11_rosparams_following_real_vehicle_micromodelv2_without_margin.csv'

In [71]:
df=pd.read_csv(file)

In [72]:
df

,/roslaunch/uris/host_2t3h1rfv8lc057037__38353,/roslaunch/uris/host_2t3h1rfv8lc057037__40633,/description,/rosversion,/run_id,/th3,/th2,/th1,/w3,/w2,/w1,/rosdistro,/margin,/gap_variant
0,http://2T3H1RFV8LC057037:38353/,http://2T3H1RFV8LC057037:40633/,following_real_vehicle_micromodelv2_without_ma...,1.14.11\n,988134e0-ee54-11eb-99f3-dca63284b91d,1.8,1.2,0.4,5.25,6.0,4.5,melodic\n,30.0,True
